In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras.backend as kb
import keras as K

from tensorflow.keras import layers
from keras.layers import Input, Flatten, Dense, Conv2D, SeparableConv2D
from keras.layers import  BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.datasets import cifar10

from keras.applications.densenet import DenseNet201
from keras.applications.xception import Xception

NUM_CLASSES = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

In [5]:
input_tensor = K.Input(shape=(32,32,3))

resized_image = K.layers.Lambda(lambda image: tf.image.resize(image, (299,299)))(input_tensor)

base_model = K.applications.Xception(include_top=False,
                                       weights="imagenet",
                                       input_tensor=resized_image,
                                    input_shape=(299,299,3)
                                    )
output = base_model.layers[-1].output
base_model = K.models.Model(inputs=input_tensor, outputs=output)

In [6]:
base_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 lambda_1 (Lambda)              (None, 299, 299, 3)  0           ['input_3[0][0]']                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['lambda_1[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']       

                                                                                                  
 block4_sepconv1_bn (BatchNorma  (None, 37, 37, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 37, 37, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 37, 37, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_se

 block7_sepconv1_act (Activatio  (None, 19, 19, 728)  0          ['add_16[0][0]']                 
 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 19, 19, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 19, 19, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 19, 19, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)       

                                                                                                  
 block9_sepconv3_bn (BatchNorma  (None, 19, 19, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_19 (Add)                   (None, 19, 19, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_18[0][0]']                 
                                                                                                  
 block10_sepconv1_act (Activati  (None, 19, 19, 728)  0          ['add_19[0][0]']                 
 on)                                                                                              
                                                                                                  
 block10_s

 alization)                                                                                       
                                                                                                  
 block12_sepconv3_act (Activati  (None, 19, 19, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 19, 19, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 19, 19, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
          

In [9]:
input_tensor = K.Input(shape=(32,32,3))

resized_image = K.layers.Lambda(lambda image: tf.image.resize(image, (299,299)))(input_tensor)

xception = K.applications.Xception(include_top = False,input_shape=(299,299,3))

In [10]:
for layer in xception.layers:
    layer.trainable = False
    
for layer in xception.layers:
    sp = '          '[len(layer.name)-9:]
    print(layer.name, sp, layer.trainable)

input_7    False
block1_conv1         False
block1_conv1_bn      False
block1_conv1_act     False
block1_conv2         False
block1_conv2_bn      False
block1_conv2_act     False
block2_sepconv1      False
block2_sepconv1_bn   False
block2_sepconv2_act  False
block2_sepconv2      False
block2_sepconv2_bn   False
conv2d_8   False
block2_pool          False
batch_normalization_8  False
add_24     False
block3_sepconv1_act  False
block3_sepconv1      False
block3_sepconv1_bn   False
block3_sepconv2_act  False
block3_sepconv2      False
block3_sepconv2_bn   False
conv2d_9   False
block3_pool          False
batch_normalization_9  False
add_25     False
block4_sepconv1_act  False
block4_sepconv1      False
block4_sepconv1_bn   False
block4_sepconv2_act  False
block4_sepconv2      False
block4_sepconv2_bn   False
conv2d_10            False
block4_pool          False
batch_normalization_10  False
add_26     False
block5_sepconv1_act  False
block5_sepconv1      False
block5_sepconv1_bn   False


In [12]:
model2 = K.models.Sequential()
model2.add(xception)
model2.add(Flatten())
model2.add( Dense(units = 10, activation= 'softmax'))
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 flatten (Flatten)           (None, 204800)            0         
                                                                 
 dense (Dense)               (None, 10)                2048010   
                                                                 
Total params: 22,909,490
Trainable params: 2,048,010
Non-trainable params: 20,861,480
_________________________________________________________________


In [15]:
opt = Adam(lr = 0.001)
model2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [18]:
model2.fit(x_train,
         y_train,
         batch_size=32,
         epochs=4,
         shuffle = True,
         validation_data = (x_test, y_test))

ResourceExhaustedError: Exception encountered when calling layer 'lambda_5' (type Lambda).

{{function_node __wrapped__ResizeBilinear_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[50000,299,299,3] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:ResizeBilinear]

Call arguments received by layer 'lambda_5' (type Lambda):
  • inputs=tf.Tensor(shape=(50000, 32, 32, 3), dtype=float32)
  • mask=None
  • training=None